In [2]:
include("functions.jl")

genetique

In [3]:
# Définition des instances de base
@static if !@isdefined(instance_list)
    const instance_list = (
        read_instance("graphs/flat300_26_0.col", 26),
        read_instance("graphs/le450_15c.col", 15),
        read_instance("graphs/dsjc125.1.col", 5),
        read_instance("graphs/dsjc125.9.col", 44),
        read_instance("graphs/dsjc250.1.col", 8),
        read_instance("graphs/dsjc250.9.col", 72),
        read_instance("graphs/dsjc250.5.col", 28),
        read_instance("graphs/dsjc1000.5.col", 86),
        read_instance("graphs/dsjc1000.5.col", 85),
        read_instance("graphs/dsjc1000.5.col", 84))
end

(Instance("flat300_26_0.col", Bool[0 1 … 0 0; 1 0 … 1 1; … ; 0 1 … 0 1; 0 1 … 1 0], 26), Instance("le450_15c.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 0; 0 0 … 0 0], 15), Instance("dsjc125.1.col", Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 5), Instance("dsjc125.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 0; 1 1 … 0 0], 44), Instance("dsjc250.1.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 1 0], 8), Instance("dsjc250.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 1; 1 1 … 1 0], 72), Instance("dsjc250.5.col", Bool[0 1 … 1 0; 1 0 … 1 0; … ; 1 1 … 0 0; 0 0 … 0 0], 28), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 86), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 85), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 84))

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [4]:
# Comparaison d'une solution aléatoire et d'une solution gloutonne
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=

5


SOLUTION ALÉATOIRE:
[4, 1, 1, 4, 3, 4, 4, 2, 1, 1, 1, 5, 5, 3, 4, 1, 5, 1, 4, 2, 4, 3, 4, 5, 1, 5, 3, 2, 3, 2, 3, 5, 5, 3, 4, 4, 3, 3, 1, 5, 5, 4, 2, 2, 3, 5, 1, 5, 1, 2, 5, 4, 4, 4, 5, 4, 4, 2, 4, 4, 1, 1, 5, 1, 5, 3, 1, 2, 3, 3, 1, 5, 2, 1, 3, 3, 1, 3, 3, 1, 5, 3, 4, 3, 5, 3, 4, 5, 1, 2, 2, 5, 2, 3, 2, 2, 4, 4, 2, 2, 4, 2, 2, 3, 4, 5, 4, 4, 5, 2, 2, 5, 3, 5, 1, 3, 2, 3, 4, 5, 2, 2, 2, 4, 2]
collisions: 

139



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [5]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0040488	0.0003621	2469.867615095831
le450_15c.col k=15	362	362.0	362	0.0043062	0.0004248	2322.2330593098327
dsjc125.1.col k=5	47	47.0	47	0.0002322	2.58e-5	43066.32213608958
dsjc125.9.col k=44	21	21.0	21	0.00031040000000000006

	3.37e-5	32216.49484536082
dsjc250.1.col k=8	88	88.0	88	0.0012573000000000003	0.00010360000000000001	7953.551260637873
dsjc250.9.col k=72	61	61.0	61	0.0013306000000000001	0.0001325	7515.406583496167
dsjc250.5.col k=28	85	85.0	85	0.0023675000000000002	0.00023050000000000002	4223.864836325237


dsjc1000.5.col k=86	378	378.0	378	0.052991500000000004	0.0039415000000000006	188.70951001575722


dsjc1000.5.col k=85	398	398.0	398	0.0527774	0.0074641	189.47504045292112


dsjc1000.5.col k=84	412	412.0	412	0.042342500000000005	0.0046977	236.1693334120564


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à réduire le plus possible le nombre de conflits par rapport à la solution courante.
   - On choisit le premier couple $(i,j)$ occasionnant la plus forte réduction de conflits.
   - Pour cela, comme suggéré par Daniel Porumbel, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [6]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = glouton(instance)
    simple_neighbor(instance,solution)
    print(solution.obj)
    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

43t: 1	conflits: 40
t: 2	conflits: 38
t: 3	conflits: 36
t: 4	conflits: 34
t: 5	conflits: 32
t: 6	conflits: 30
t: 7	conflits: 28
t: 8	conflits: 27
t: 9	conflits: 26
t: 10	conflits: 25
t: 11	conflits: 24
t: 12	conflits: 23
t: 13	conflits: 22
t: 14	conflits: 21
t: 15	conflits: 20
t: 16	conflits: 19
t: 17	conflits: 18
t: 18	conflits: 17
t: 19	conflits: 16
t: 20	conflits: 15
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15


# 3) Métaheuristique proposée

On choisit d'écrire un algorithme mémétique:

## 0 - Règle de sélection

On utilise la règle de la roue de la fortune, en utilisant comme poids l'objectif dans l'opérateur d'élimination et l'inverse de l'objectif dans la fonction de reproduction.

## 1 - Fonction de reproduction

On génère λ enfants, chacun à partir de deux parents sélectionnés avec la règle ci-dessus, en leur appliquant l'opérateur de croisement.\
Afin de garantir la diversité des solutions, si l'enfant est trop proche d'une solution existante (plus proche que le seuil de rejet, voir opérateur d'élimination), il est rejeté.

### Opérateur de croisement

Comme suggéré par Daniel Porumbel, on choisit un opérateur de croisement qui copie des classes de couleurs entières des parents dans l'enfant (sur les nœuds où l'enfant n'a pas encore de couleur définie), dans l'ordre décroissant de taille de la classe, indifféremment de leur origine parentale.

## 2 - Opérateur d'intensification

On effectue une recherche locale simple, en utilisant la structure de voisinage simple `simple_neighbor` citée ci-dessus.\
On interrompt la recherche locale dès que l'on atteint un minimum local ou un plateau (i.e. si la valeur de l'objectif n'a pas changé). Dans tous les cas, la recherche locale se termine si l'on atteint le nombre maximal d'itérations autorisé.

## 3 - Fonction d'élimination

Comme proposé par Daniel Porumbel, on calcule la distance entre chaque individu de la population.
 - Si la distance minimale est inférieure au seuil de rejet, on élimine la solution de moins bon objectif.
 - Sinon, on utilise la règle de sélection pour obtenir une solution, on calcule son voisin le plus proche, et on élimine celle de moins bon objectif.

### Calcul de distance entre deux solutions

Comme Daniel Porumbel l'a fait remarquer, la distance entre deux solutions n'est pas simplement le nombre de couleurs différentes, car la solution est définie à une permutation des couleurs près. Pour calculer la distance, il est donc nécessaire de déterminer la permutation minimisant le nombre de couleurs différentes entre deux solutions, donc maximisant le nombre de couleurs identiques:

$\displaystyle{\max_{\text{perm}} \sum_{c} S(c,\text{perm}(c))}$

Où $S$ est la matrice de similarité, i.e. $S(a,b)$ est le nombre de nœuds $i$ pour lesquels $x_1(i)=a$ et $x_2(i)=b$.

Le calcul de la meilleure permutation est un problème NP-complet, ce qui exclut l'utilisation d'un algorithme exact pour la déterminer. On écrit donc une heuristique:

Les cases de $S$ contenant les plus grandes valeurs ont de grandes chances d'être atteintes par la permutation. On définit donc la permutation comme suit:
 - $(a,b) = \argmax(S)$
 - $\text{perm}(b) = a$
 - vider la colonne $b$ (correspondant à $\text{perm}(b) = u$, car perm est une fonction) ainsi que la ligne $a$ (correspondant à $\text{perm}(v) = a$, car perm est injective)
 - recommencer

Cette heuristique a toutes les chances de fournir la permutation optimale si les solutions sont effectivement proches, ce qui donne une matrice $S$ avec une case de valeur nettement plus grande que les autres de sa ligne/colonne, donc détectable par l'heuristique. Cela tombe bien, car il est important d'avoir une bonne estimation de la distance pour des solutions proches, alors que pour des solutions éloignées, cela est moins critique.

Test: Tabou

In [7]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = glouton(instance)

    for t = 1:100
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 43
t: 2	conflits: 40
t: 3	conflits: 38
t: 4	conflits: 36
t: 5	conflits: 34
t: 6	conflits: 32
t: 7	conflits: 30
t: 8	conflits: 28
t: 9	conflits: 27
t: 10	conflits: 26
t: 11	conflits: 25
t: 12	conflits: 24
t: 13	conflits: 23
t: 14	conflits: 22
t: 15	conflits: 21
t: 16	conflits: 20
t: 17	conflits: 19
t: 18	conflits: 18
t: 19	conflits: 17
t: 20	conflits: 16
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15
t: 26	conflits: 15
t: 27	conflits: 15
t: 28	conflits: 15
t: 29	conflits: 15
t: 30	conflits: 15
t: 31	conflits: 15
t: 32	conflits: 15
t: 33	conflits: 15
t: 34	conflits: 15
t: 35	conflits: 15
t: 36	conflits: 15
t: 37	conflits: 15
t: 38	conflits: 15
t: 39	conflits: 15
t: 40	conflits: 15
t: 41	conflits: 15
t: 42	conflits: 15
t: 43	conflits: 15
t: 44	conflits: 15
t: 45	conflits: 15
t: 46	conflits: 15
t: 47	conflits: 15
t: 48	conflits: 15
t: 49	conflits: 15
t: 50	conflits: 15
t: 51	conflits: 15
t: 52	conflits: 15
t: 53	conflits: 15
t:

Algo mémétique

In [7]:
begin
    start_time = time()
    instance = instance_list[3]
    instopt = genetique(instance,25,10,10,80) 
    print("Nombre de collisions : ",instopt.obj, "\n")
    print("temps d'execution : ",trunc(100*(time()-start_time))/100,"s")
end

Nombre de collisions : 4
temps d'execution : 1.81s

┌ Info: it: 80	temps:0.54s	conflits: [6, 7, 6, 6, 5, 4, 6, 12, 8, 8, 9, 8, 10, 10, 12, 4, 7, 10, 7, 7, 11, 12, 8, 9, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


# 4) Tests

On effectue des tests, les résultats de tous les tests peuvent être retrouvés dans les fichiers temps_d_executions et val_fonc-obj. Les anciens tests ont un paramètre de plus que les nouveaux, l'un ayant été supprimé. On peut voir la date ou le test à été effectué.

On détaille plus bas les choix des paramètres, les tests nous ont aidés à avoir une idée de valeur raisonnable pour les choix des paramètres.

Cellule d'ouverture des streams, à activer avec précaution

In [8]:
iotemps = open("temps_d_executions", "w")
ioobj = open("val_fonc-obj","w")

write(iotemps, "instance:, ", "graphs/flat300_26_0.col k=26, ")
write(iotemps,"graphs/le450_15c.col k=15, ")
write(iotemps,"graphs/dsjc125.1.col k=5, ")
write(iotemps,"graphs/dsjc125.9.col k=44, ")
write(iotemps,"graphs/dsjc250.1.col k=8, ")
write(iotemps,"graphs/dsjc250.9.col k=72, ")
write(iotemps,"graphs/dsjc250.5.col k=28, ")
write(iotemps,"graphs/dsjc1000.5.col k=86, ")
write(iotemps,"graphs/dsjc1000.5.col k=85, ")
write(iotemps,"graphs/dsjc1000.5.col k=84, \n")

write(ioobj, "instance:, ", "graphs/flat300_26_0.col k=26, ")
write(ioobj,"graphs/le450_15c.col k=15, ")
write(ioobj,"graphs/dsjc125.1.col k=5, ")
write(ioobj,"graphs/dsjc125.9.col k=44, ")
write(ioobj,"graphs/dsjc250.1.col k=8, ")
write(ioobj,"graphs/dsjc250.9.col k=72, ")
write(ioobj,"graphs/dsjc250.5.col k=28, ")
write(ioobj,"graphs/dsjc1000.5.col k=86, ")
write(ioobj,"graphs/dsjc1000.5.col k=85, ")
write(ioobj,"graphs/dsjc1000.5.col k=84, \n")

29

Test glouton

In [9]:
begin
    write(ioobj, "Glouton, ")
    write(iotemps, "Glouton, ")
    for instance in instance_list
        start_time = time()
        solution = glouton(instance)
        write(ioobj, repr(nbr_collision(instance,solution)), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

1

Tests de l'algorithme mémétique avec différents paramètres.

In [10]:
begin
    write(ioobj, "Genetique 10 10 k/2 10, ")
    write(iotemps, "Genetique 10 10 k/2 10, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,10,10,div(instance.k,2),10)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

┌ Info: it: 10	temps:1.56s	conflits: [132, 131, 140, 131, 127, 136, 133, 128, 131, 123]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.21s	conflits: [266, 263, 258, 260, 268, 277, 272, 274, 276, 266]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.03s	conflits: [11, 8, 8, 12, 8, 9, 8, 7, 7, 7]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:0.83s	conflits: [17, 11, 10, 12, 11, 14, 11, 11, 12, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.2s	conflits: [32, 38, 27, 26, 28, 31, 35, 40, 40, 39]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:5.81s	conflits: [34, 38, 37, 33, 40, 33, 30, 31, 33, 36]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.0s	conflits: [51, 49, 47, 50, 47, 49, 43, 47, 44, 49]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:49.88s	conflits: [258, 269, 265, 255, 236, 250, 238, 258, 277, 233]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:50.81s	conflits: [263, 271, 281, 257, 286, 274, 260, 284, 280, 297]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:51.91s	conflits: [280, 298, 309, 301, 299, 290, 252, 266, 285, 299]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


1

In [11]:
begin
    write(ioobj, "Genetique 10 10 k 10, ")
    write(iotemps, "Genetique 10 10 k 10, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,10,10,instance.k,10)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

┌ Info: it: 10	temps:1.03s	conflits: [134, 122, 123, 127, 152, 117, 116, 124, 127, 144]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.13s	conflits: [299, 255, 267, 268, 288, 267, 258, 264, 273, 264]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.03s	conflits: [11, 12, 8, 8, 9, 13, 7, 11, 12, 9]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:0.16s	conflits: [26, 23, 28, 22, 26, 23, 19, 26, 24, 23]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.16s	conflits: [31, 28, 36, 34, 34, 34, 35, 34, 24, 37]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:0.75s	conflits: [58, 58, 55, 61, 69, 66, 64, 60, 68, 72]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.91s	conflits: [47, 44, 49, 45, 45, 55, 48, 61, 54, 47]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:52.96s	conflits: [291, 302, 260, 285, 260, 280, 242, 255, 266, 264]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:52.12s	conflits: [308, 276, 278, 283, 286, 258, 265, 248, 284, 261]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:52.76s	conflits: [319, 317, 311, 318, 303, 269, 266, 292, 301, 322]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


1

In [12]:
begin
    write(ioobj, "Genetique 10 10 k/4 10, ")
    write(iotemps, "Genetique 10 10 k/4 10, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,10,10,div(instance.k,4),10)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

┌ Info: it: 10	temps:1.16s	conflits: [138, 122, 145, 135, 131, 132, 151, 123, 127, 132]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.18s	conflits: [260, 243, 253, 255, 250, 251, 265, 251, 256, 249]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:0.03s	conflits: [8, 11, 7, 10, 7, 10, 8, 9, 8, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.54s	conflits: [10, 9, 11, 9, 6, 11, 10, 8, 11, 11]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.16s	conflits: [30, 31, 30, 32, 32, 26, 26, 32, 38, 33]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:7.61s	conflits: [38, 38, 39, 32, 35, 44, 36, 35, 33, 35]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.95s	conflits: [53, 72, 46, 52, 58, 57, 51, 59, 47, 41]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:55.78s	conflits: [284, 294, 287, 262, 251, 249, 311, 301, 282, 304]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:56.47s	conflits: [299, 276, 272, 271, 282, 289, 309, 275, 253, 279]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:58.05s	conflits: [306, 280, 281, 279, 280, 310, 278, 301, 275, 286]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


1

In [13]:
begin
    write(ioobj, "Genetique 10 10 k/3 10, ")
    write(iotemps, "Genetique 10 10 k/3 10, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,10,10,div(instance.k,3),10)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

┌ Info: it: 10	temps:1.4s	conflits: [130, 141, 128, 135, 140, 137, 141, 132, 142, 144]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.38s	conflits: [260, 248, 256, 235, 258, 249, 239, 249, 257, 240]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.03s	conflits: [8, 9, 9, 9, 11, 10, 11, 10, 9, 9]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:1.55s	conflits: [14, 12, 13, 13, 9, 13, 14, 12, 12, 12]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:0.15s	conflits: [46, 30, 30, 35, 33, 31, 30, 28, 29, 34]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:7.81s	conflits: [31, 33, 32, 32, 28, 30, 35, 32, 32, 37]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:1.08s	conflits: [50, 55, 45, 59, 53, 49, 45, 58, 54, 59]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 10	temps:48.51s	conflits: [244, 252, 275, 241, 263, 277, 247, 248, 263, 270]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:60.88s	conflits: [290, 279, 264, 282, 254, 272, 279, 251, 274, 289]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 10	temps:55.0s	conflits: [290, 288, 291, 286, 309, 265, 274, 303, 250, 285]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


1

Cellule de fermeture des streams, à activer une fois que tout est écrit

In [14]:
close(iotemps)
close(ioobj)

Les tests effectués indiquent que modifier certains paramètres est très coûteux en temps pour une amélioration assez faible de la fonction objective. Je vais donc détailler notre choix de paramètres.

Pop size : ne pas avoir trop haut, 10 semble raisonnable

Nbrchildren : paramètre très couteux à avoir haut, à garder assez faible, 10 semble raisonnable

DistTHR : paramètre à garder imperativement faible pour les instances de petites taille, sinon le faire grandir n'est pas très coûteux, Je pense que l'avoir à 3 ou 5 pour les petites instances sinon l'avoir de l'ordre de grandeur de k/2 ne me semble pas absurde

Local Maxit (Parametre supprimé, utile si l'on veux comprendre les anciennes versions de l'algorithme): parametre peu coûteux à garder élevé : 200 est un bon ordre de grandeur

Maxit : nombre d'itération de l'algo

Code pour calculer le récapitulatif numérique de notre algorithme.

In [ ]:
begin
    display("Genetique 10 10 k/2 10, ")
    display("Genetique 10 10 k/2 10, ")
    for instance in instance_list
        min = 0
        max = 10000000
        start_time = time()
        t
        for i in 1:10
            solution = genetique(instance,10,10,div(instance.k,2),10)
            write(ioobj, repr(solution.obj), ", ")
        end
    end_time = time()
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

In [8]:
instance = instance_list[3]
# Number of times to run the function
num_tests = 10

# Initialize variables to store min, max, and sum of results
min_result = Inf
max_result = -Inf
total_result = 0

start_time = time()
# Run the function 10 times and calculate min, max, and sum
for _ in 1:num_tests
    solution = genetique(instance,10,10,div(instance.k,2),20)
    min_result = min(min_result, solution.obj)
    max_result = max(max_result, solution.obj)
    total_result += solution.obj
end

# Calculate average result
average_result = total_result / num_tests

# Print the results
println("Average time: ", trunc(100*(time()-start_time))/(100*num_tests),"s")
println("Minimum result: ", min_result)
println("Maximum result: ", max_result)
println("Average result: ", average_result)

┌ Info: it: 20	temps:0.13s	conflits: [7, 7, 8, 9, 10, 11, 10, 9, 7, 11]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 20	temps:0.11s	conflits: [11, 7, 8, 5, 6, 5, 7, 7, 6, 4]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 20	temps:0.1s	conflits: [13, 10, 7, 7, 6, 7, 6, 8, 7, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 20	temps:0.09s	conflits: [8, 6, 6, 7, 5, 7, 9, 8, 11, 7]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 20	temps:0.1s	conflits: [7, 8, 10, 7, 13, 9, 13, 8, 11, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 20	temps:0.08s	conflits: [8, 9, 8, 9, 11, 9, 11, 9, 11, 11]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


┌ Info: it: 20	temps:0.1s	conflits: [6, 6, 6, 7, 9, 6, 9, 10, 9, 10]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 20	temps:0.1s	conflits: [10, 8, 10, 10, 8, 10, 9, 9, 10, 9]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332


Average time: 0.25s
Minimum result: 4.0
Maximum result: 8.0
Average result: 6.6


┌ Info: it: 20	temps:0.08s	conflits: [9, 11, 9, 12, 8, 10, 8, 8, 8, 9]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
┌ Info: it: 20	temps:0.13s	conflits: [10, 11, 9, 10, 7, 12, 14, 11, 10, 13]
└ @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:332
